In [1]:
from openai import OpenAI
from human_eval.data import read_problems, write_jsonl
from human_eval.evaluation import evaluate_functional_correctness
import matplotlib.pyplot as plt
import os

Локальный сервер LM Studio + default API key

In [2]:
openai = OpenAI(base_url='http://127.0.0.1:1234/v1', api_key="lm-studio")
model = "qwen2.5-3b"

In [3]:
problems = read_problems()

Температуры для сэтов генерации

In [4]:
temperatures = [0.1, 0.3, 0.5, 0.7, 0.9]

Количество токенов для одной генерации и количество генераций на задачу

In [5]:
results = {}
num_samples_per_task = 25
max_tokens = 512

Генерация ответа

In [6]:
def generate_code(openai, model, prompt, temperature, num_samples, max_tokens):
    responses = []
    for _ in range(num_samples):
        completion = openai.completions.create(
            model=model,
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=temperature
        )
        responses.append(completion.choices[0].text)
    return responses

Сбор jsonl для одной температуры (поочередно) с генерацией по указанному количеству samples

In [7]:
for temp in temperatures:
    samples_file = f"samples_temp_{temp}.jsonl"
    if not os.path.exists(samples_file):
        print(f"Генерируем {num_samples_per_task} выборок при temperature={temp}...")
        samples = []
        for task_id, task in problems.items():
            prompt = task["prompt"]
            completions = generate_code(openai, model, prompt, temperature=temp, num_samples=num_samples_per_task, max_tokens=max_tokens)
            for comp in completions:
                samples.append({"task_id": task_id, "completion": comp})
        write_jsonl(samples_file, samples)
        print(f"Сохранено {len(samples)} выборок в файл {samples_file}")
    else:
        print(f"Файл {samples_file} уже существует, используем его.")

Оценка полученного набора генераций по pass@25 через evaluate_function_correctness

In [ ]:
for temp in temperatures:
    samples_file = f"samples_temp_{temp}.jsonl"
    eval_result = evaluate_functional_correctness(samples_file, k=[25])
    pass_at_1 = eval_result.get('pass@25', None)
    results[temp] = pass_at_25
    print(f"temperature={temp}: pass@1 = {pass_at_25}")

Построение простого графика для визуальной оценки

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(list(results.keys()), list(results.values()), marker='o')
plt.xlabel("Temperature")
plt.ylabel("pass@25")
plt.grid(True)
plt.show()